### Домашнее задание по NLP # 1
Классификация по тональности
В этом домашнем задании вам предстоит классифицировать по тональности отзывы на банки с сайта banki.ru.

Данные содержат непосредственно тексты отзывов, некоторую дополнительную информацию, а также оценку по шкале от 1 до 5.

Тексты хранятся в json-ах в массиве responses.

### Часть 1. Анализ текстов [40/100]¶
1. Посчитайте количество отзывов в разных городах и на разные банки  
2. Постройте гистограмы длин слов в символах и в словах
3. Найдите 10 самых частых:
   - слов
   - слов без стоп-слов
   - лемм
   - существительных
4. Постройте кривые Ципфа и Хипса
5. Ответьте на следующие вопросы:
   - какое слово встречается чаще, "сотрудник" или "клиент"?
   - сколько раз встречается слова "мошенничество" и "доверие"?
6. В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру $tf-idf$, для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)




In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline


import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer

Populating the interactive namespace from numpy and matplotlib


In [2]:
# читаем файл json зипованный в формат bz2
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [03:12, 1044.55it/s]


In [49]:
# создаем списки со всеми вхождениями банков и городов (из них потом будем считать)

cities = []
for i in range(0, len(responses)):
    city = responses[i]['city']
    cities.append(city)
    
banks = []
for i in range(0, len(responses)):
    bank = responses[i]['bank_name']
    banks.append(bank)

#### Посчитайте количество отзывов в разных городах и на разные банки

In [50]:
qty_cities = Counter(cities)

In [51]:
qty_cities.most_common(200)

[('г. Москва', 55354),
 (None, 15174),
 ('г. Санкт-Петербург', 14342),
 ('г. Екатеринбург', 2337),
 ('г. Новосибирск', 2300),
 ('г. Нижний Новгород', 1976),
 ('г. Ростов-на-Дону', 1891),
 ('г. Самара', 1731),
 ('г. Челябинск', 1507),
 ('г. Казань', 1440),
 ('г. Краснодар', 1262),
 ('г. Уфа', 1246),
 ('г. Красноярск', 1236),
 ('г. москва', 1217),
 ('г. Воронеж', 1210),
 ('г. Пермь', 1163),
 ('г. Волгоград', 1014),
 ('г. Омск', 989),
 ('г. Саратов', 933),
 ('г. Иркутск', 749),
 ('г. Калининград', 746),
 ('г. Ставрополь', 696),
 ('г. Хабаровск', 681),
 ('г. Тюмень', 656),
 ('г. Владивосток', 637),
 ('г. Ярославль', 609),
 ('г. Барнаул', 609),
 ('г. Тула', 586),
 ('г. Томск', 569),
 ('г. Рязань', 548),
 ('г. Ульяновск', 500),
 ('г. Тольятти', 473),
 ('г. Липецк', 461),
 ('г. Иваново', 458),
 ('г. Ижевск', 454),
 ('г. Тверь', 447),
 ('г. Мурманск', 434),
 ('г. Пенза', 433),
 ('г. Оренбург', 431),
 ('г. Чебоксары', 392),
 ('г. Кемерово', 390),
 ('г. Белгород', 387),
 ('г. Магнитогорск', 379)

In [52]:
qty_banks = Counter(banks)
qty_banks.most_common(20)

[('Сбербанк России', 26327),
 ('Альфа-Банк', 10224),
 ('ВТБ 24', 8185),
 ('Русский Стандарт', 7943),
 ('Хоум Кредит Банк', 7549),
 ('Тинькофф Банк', 5387),
 ('Национальный Банк «Траст»', 4607),
 ('Ренессанс Кредит', 3849),
 ('Связной Банк', 3775),
 ('ОТП Банк', 3237),
 ('Восточный Экспресс Банк', 3138),
 ('Райффайзенбанк', 3057),
 ('Московский Кредитный Банк', 2944),
 ('Ситибанк', 2848),
 ('Авангард', 2733),
 ('Промсвязьбанк', 2586),
 ('ЮниКредит Банк', 2488),
 ('Банк Москвы', 2456),
 ('Ханты-Мансийский банк Открытие', 2158),
 ('МТС Банк', 1870)]

In [3]:
# создадим функцию которая чистит текст (только русские слова, перевод в нижний регистр, )
def clean_text(text):
    text = re.sub('[^А-Яа-яё]', ' ', text)
    text = text.lower()
    return text

In [4]:
# создадим корпус из отзывов
corpus = []
for i in range(0, len(responses)):
    text = clean_text(responses[i]['text'])
    corpus.append(text)

In [5]:
corpus[0]

'добрый день  я не являюсь клиентом банка и поручителем по кредитному договору  а также не являюсь каким либо другим лицом  письменно  оформившим отношения с банком по поводу урегулирования чьей либо  задолженности   начиная с марта      года начали приходить бесконечные письма из ооо  примо коллект на мой адрес  город москва  уваровский переулок  дом     квартира      с угрозами о возбуждении уголовного дела в отношении гражданина филиппова эдуарда владимировича  который уклоняется от уплаты взятых им кредитов  договор         от            года и договор           от            года  со всей ответственностью  хочу вас заверить  что вышеуказанный гражданин  которого вы разыскиваете  мне не знаком и никогда в моем адресе не был зарегистрирован  каким образом вы не удостоверившись в подлинности его документов оформили на его имя кредитный договор  мне по меньшей мере не понятно   и почему по вашей милости я должна переживать и бояться за себе и свое имущество  письма доставляют мне серь

In [3]:
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
pymorphy2_analyzer = MorphAnalyzer()

In [14]:
def text_preprocessing(review):
    # только текст
    review = re.sub('[^А-Яа-яё]', ' ', review)
    
    # понижение регистра
    review = review.lower()
    
    # токенизация
    tokens = nltk.word_tokenize(review)
    
    # удаление стоп слов
    review = [word for word in tokens if word not in mystopwords]
    
    # лемматизация
    review = [pymorphy2_analyzer.parse(word)[0].normal_form for word in review]
    review = ' '.join(review)
    
    return review

In [16]:
%%time
corpus = []
for i in range(0, len(responses)):
    review = text_preprocessing(responses[i]['text'])
    corpus.append(review)

Wall time: 1h 38min 29s


In [17]:
corpus[1]

'добрый день являться держатель зарплатный карта выдать сбербанк россия год сменить фамилия предоставить банк документ копия новое паспорт свидетельство брак перевыпустить карта фамилия муж токарев мария александр девичий секлетина возникнуть необходимость подключить услуга сбербанк онлайн получиться главный страница отображаться мой карта связаться технический специалист отделение сбербанк который обслуживаться пояснить база банка ещё девичий фамилия секлетина предложить выслать скан необходимый документ выслать оставить контакт звонок дождаться число з некоторый время звонить сам объяснять ситуация пробивать база числиться секлетина хотя карта перевыпустить фамилия токарев связь истечение срок обслуживание высылать документ копия паспорт свидетельство брак результат ноль просить помочь разобраться ситуация уважение токарев мария александр'

#### Найдите 10 самых частых:
    - слов
    - слов без стоп-слов
    - лемм
    - существительных

In [11]:
lemmata = [lemma for text in corpus for lemma in text.split()]

In [33]:
len(lemmata)

35354972

In [34]:
cnt = Counter(lemmata)

In [107]:
cnt.most_common(10)

[('в', 1330313),
 ('и', 1058997),
 ('не', 852440),
 ('на', 707803),
 ('что', 640057),
 ('я', 577500),
 ('с', 463030),
 ('по', 372759),
 ('мне', 320958),
 ('банка', 275906)]

In [12]:
# улалим стоп слова
mystopwords = stopwords.words('russian')

In [13]:
def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

In [14]:
corpus_without_stop = []
for i in lemmata:
    if i not in mystopwords:
        corpus_without_stop.append(i)
#     text = remove_stopwords(i, mystopwords = mystopwords)
#     corpus_without_stop.append(text)

KeyboardInterrupt: 

In [13]:
len(lemmata)

35354972

In [14]:
len(corpus_without_stop)

22290441

In [130]:
# lemmata_without_stop = [lemma for text in corpus_without_stop for lemma in text.split()]

In [139]:
cnt_without_stop = Counter(corpus_without_stop)
cnt_without_stop.most_common(10)

[('банка', 275906),
 ('банк', 215425),
 ('это', 183365),
 ('деньги', 131201),
 ('карту', 105829),
 ('карты', 101821),
 ('кредит', 81135),
 ('день', 76552),
 ('т', 75826),
 ('банке', 72169)]

In [155]:
# делаем лемматизацию слов
m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return " ".join(m.lemmatize(text)).strip()  
    except:
        return ""

In [168]:
corpus_lemma = []
for i in range(0, len(corpus_without_stop)):
    text = lemmatize(corpus_without_stop[i], mystem=m)
    corpus_lemma.append(text)

KeyboardInterrupt: 

In [ ]:
cnt_lemma = Counter(corpus_lemma)
cnt_lemma.most_common(10)

In [170]:
pip install pymorphy2

  Stored in directory: C:\Users\38E2~1\AppData\Local\pip\Cache\wheels\9b\04\dd\7daf4150b6d9b12949298737de9431a324d4b797ffd63f526e
Successfully built docopt
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pymorphy2 import MorphAnalyzer

In [16]:
pymorphy2_analyzer = MorphAnalyzer()

In [25]:
corpus_lemma = []
for i in range(0, len(corpus_without_stop)):
    text = pymorphy2_analyzer.parse(corpus_without_stop[i])[0].normal_form
    corpus_lemma.append(text)

In [26]:
cnt_lemma = Counter(corpus_lemma)
cnt_lemma.most_common(10)

[('банка', 399725),
 ('карта', 334999),
 ('банк', 268851),
 ('это', 209775),
 ('деньга', 179819),
 ('день', 170149),
 ('год', 169226),
 ('мой', 162067),
 ('кредит', 156176),
 ('который', 149017)]

4. Постройте кривые Ципфа и Хипса

In [28]:
freqs = list(cnt_lemma.values())
freqs = sorted(freqs, reverse = True)

fig, ax = plt.subplots()
ax.plot(freqs[:300], range(300))
plt.show()

KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm

cnt = Counter()
n_words = []
n_tokens = []
tokens = []
for index, row in tqdm(df.iterrows(), total = len(df)):
    tokens = word_tokenize(row['text'])
    cnt.update([token for token in tokens if token not in punctuation])
    n_words.append(len(cnt))
    n_tokens.append(sum(cnt.values()))cnt_lemma = list(corpus_lemma.values())
    
fig, ax = plt.subplots()
ax.plot(n_tokens, n_words)
plt.show()

5. Ответьте на следующие вопросы:
   - какое слово встречается чаще, "сотрудник" или "клиент"?
   - сколько раз встречается слова "мошенничество" и "доверие"?

In [39]:
if cnt_lemma['сотрудник'] > cnt_lemma['клиент']:
    print('Упоминаний слова сотрудник больше чем слова клиент')
else:
    print('Упоминаний слова клиент больше чем слова сотрудник')

Упоминаний слова клиент больше чем слова сотрудник


In [43]:
m = cnt_lemma['мошенничество']
d = cnt_lemma['доверие']
print(f'Слово мошенничество встречается {m} раз')
print(f'Слово доверие встречается {d} раз')

Слово мошенничество встречается 3244 раз
Слово доверие встречается 2108 раз


6. В поле "rating_grade" записана оценка отзыва по шкале от 1 до 5. Используйте меру $tf-idf$, для того, чтобы найти ключевые слова и биграмы для положительных отзывов (с оценкой 5) и отрицательных отзывов (с оценкой 1)

- выделить в отдельные списки положительные и отрицательные списки
- найти ключевые слова

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
# записываем отдельно все положительные отзывы
positive = []
for i in range(0, len(responses)):
    if responses[i]['rating_grade'] == 5:
        positive.append(responses[i]['text'])

In [32]:
# записываем отдельно все негативные отзывы
negative = []
for i in range(0, len(responses)):
    if responses[i]['rating_grade'] == 1:
        negative.append(responses[i]['text'])

In [55]:
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']

In [72]:
# создаем функцию которая
# оставляет только русский текст
# понижает регистр
# токенизирует документ по словам
# удаляет стоп слова
def only_text(text_comm):
    text = re.sub('[^А-ЯЁа-яё]', ' ', text_comm).lower().split()
    text = [word for word in text if word not in mystopwords]
    return text

In [74]:
pos = []
for i in range(0, len(positive)):
    text = only_text(positive[i])
    pos.append(text)

In [77]:
neg = []
for i in range(0, len(negative)):
    text = only_text(negative[i])
    neg.append(text)

In [80]:
# сделаем лемматизацию с помощью pymorphy
pymorphy2_analyzer = MorphAnalyzer()

In [89]:
pos_lemm = []
for i in range(0, len(pos)):
    for j in range(0, len(pos[i])):
        word = pymorphy2_analyzer.parse(pos[i][j])[0].normal_form
        pos_lemm.append(word)

In [93]:
%%time
neg_lemm = []
for i in range(0, len(neg)):
    for j in range(0, len(neg[i])):
        word = pymorphy2_analyzer.parse(neg[i][j])[0].normal_form
        neg_lemm.append(word)

Wall time: 45min 21s


In [90]:
vec = TfidfVectorizer()
X_pos = vec.fit_transform(pos_lemm)

In [92]:
idx = np.ravel(X_pos.sum(axis=0).argsort(axis=1))[::-1][:10]
top_10_words = np.array(vec.get_feature_names())[idx].tolist()
top_10_words

['банка',
 'банк',
 'карта',
 'очень',
 'год',
 'кредит',
 'день',
 'клиент',
 'отделение',
 'который']

In [94]:
vec = TfidfVectorizer()
X_neg = vec.fit_transform(neg_lemm)

In [95]:
idx = np.ravel(X_neg.sum(axis=0).argsort(axis=1))[::-1][:10]
top_10_words = np.array(vec.get_feature_names())[idx].tolist()
top_10_words

['банка',
 'карта',
 'банк',
 'деньга',
 'день',
 'мой',
 'год',
 'отделение',
 'который',
 'счёт']

In [ ]:
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
pymorphy2_analyzer = MorphAnalyzer()

In [ ]:
def text_preprocessing(review):
    # только текст
    review = re.sub('[^А-Яа-яё]', ' ', review)
    
    # понижение регистра
    review = text.lower()
    
    # токенизация
    tokens = nltk.word_tokenize(review)
    
    # удаление стоп слов
    review = [word for word in tokens if word not in mystopwords]
    
    # лемматизация
    review = [pymorphy2_analyzer.parse(word)[0].normal_form for word in review]
    review = ' '.join(review)
    
    return review

In [55]:
responses[1]

{'city': 'г. Новосибирск',
 'rating_not_checked': False,
 'title': 'Не могу пользоваться услугой Сбербанк он-лайн',
 'num_comments': 0,
 'bank_license': 'лицензия № 1481',
 'author': 'Foryou',
 'bank_name': 'Сбербанк России',
 'datetime': '2015-06-08 11:09:57',
 'text': 'Доброго дня! Являюсь держателем зарплатной карты , выданной Сбербанком России. В 2008 году сменила фамилию, предоставила в банк \xa0документы (копию нового паспорта, свидетельства о браке), мне перевыпустили карту на фамилию мужа ТОКАРЕВА МАРИЯ АЛЕКСАНДРОВНА (девичья \xa0- СЕКЛЕТИНА). \xa0 Возникла необходимость подключить услугу Сбербанк онлайн, НЕ ПОЛУЧИЛОСЬ (на главной странице на отображались мои карты). Связалась с техническим специалистом отделения Сбербанка в котором обслуживалась. Он пояснил, что в базе банка, я все ещё под девичьей фамилией СЕКЛЕТИНА. Предложил выслать сканы необходимых документов. Выслала, оставив контакты, звонка так и не дождалась. Ч/з некоторое время звоню сама, объясняю ситуацию, меня про

Часть 2. Тематическое моделирование [20/100]
Постройте несколько тематических моделей коллекции документов с разным числом тем. Приведите примеры понятных (интерпретируемых) тем.
Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить?
Эта часть задания может быть сделана с использованием gensim.

Часть 3. Классификация текстов [40/100]¶
Сформулируем для простоты задачу бинарной классификации: будем классифицировать на два класса, то есть, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5).

Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой 5 (значение N1 и N2 – на ваше усмотрение). Используйте sklearn.model_selection.train_test_split для разделения множества отобранных документов на обучающее и тестовое.
Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных $n$-грам.
Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков:
1-ый вариант: $tf-idf$ преобразование (sklearn.feature_extraction.text.TfidfTransformer) и сингулярное разложение (оно же – латентый семантический анализ) (sklearn.decomposition.TruncatedSVD),
2-ой вариант: тематические модели LDA (sklearn.decomposition.LatentDirichletAllocation).
Используйте accuracy и F-measure для оценки качества классификации.

Ниже написан примерный Pipeline для классификации текстов.

Эта часть задания может быть сделана с использованием sklearn.